# ch 12_8 naive bayes를 이용한 텍스트 분류

이번 챕터에서는 scikit learn에 내장된 NaiveBayesClassifier를 이용해서 실제 텍스트를 분류해보고, 성능을 측정해보겠습니다. 

## 데이터 셋 준비 

In [2]:
import pandas as pd

train_df = pd.read_csv("./data/naver_reviews_train.csv")
test_df = pd.read_csv("./data/naver_reviews_test.csv")

### 결측치 확인 및 제거

In [3]:
train_df.isnull().sum()

document    5
label       0
dtype: int64

In [4]:
test_df.isnull().sum()

document    3
label       0
dtype: int64

In [5]:
train_df = train_df.dropna()

In [6]:
test_df = test_df.dropna()

### 토큰화

In [7]:
from konlpy.tag import Komoran

komoran = Komoran()

In [8]:
from tqdm import tqdm

tqdm.pandas()
train_df["tokens"] = train_df["document"].progress_apply(lambda x: komoran.morphs(x))
test_df["tokens"] = test_df["document"].progress_apply(lambda x: komoran.morphs(x))

100%|███████████████████████████████████████████████████████████████████████| 49997/49997 [00:16<00:00, 3061.69it/s]


### CountVectorizer를 이용한 벡터화
Naive Bayes를 학습시키기 위해서는 문서 내 특정 단어의 빈도 수를 세어주어야 합니다. 이를 CountVectorizer를 이용해서 구현해보겠습니다.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

def dummy_fun(doc):
    return doc

vectorizer = CountVectorizer(
    analyzer="word",
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

In [10]:
vectorizer.fit(train_df["tokens"])
x_train = vectorizer.transform(train_df["tokens"])
y_train = train_df["label"]
x_test = vectorizer.transform(test_df["tokens"])
y_test = test_df["label"]

## Naive Bayes Classifier 학습

scikit learn에 내장된 MultinomialNB를 이용하여 Naive Bayes Classifier를 학습시켜 보겠습니다. MultinomialNB를 사용하는 이유는 단어의 등장 빈도 수를 이용하여 분류하기 때문입니다. 먼저 학습시에 교차 검증으로 정확도를 측정하고, 전체 데이터 셋으로 다시 학습시킨 뒤, 테스트 셋에 대하여 정확도를 측정해보겠습니다.

In [20]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [26]:
from sklearn.naive_bayes import MultinomialNB 

model = MultinomialNB()
cross_val_scores = cross_val_score(model, x_train, y_train, cv=stratified_kfold)
print("교차 검증 정확도:", cross_val_scores)

교차 검증 정확도: [0.82666089 0.82946098 0.83222774 0.83076103 0.83149438]


In [27]:
model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

In [28]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("테스트 셋 정확도:", accuracy)

테스트 셋 정확도: 0.82406944416665


## 정리

이번 챕터에서는 scikit learn에 내장된 MultinomialNB 클래스를 이용하여 텍스트 분류 모델을 학습시켜 보았습니다. 그 결과 82%로 상당히 준수한 성능이 나왔습니다. LSTM을 이용한 딥러닝 모델도 85% 정도 성능을 내는 것을 생각해보면 굉장히 가성비가 좋은 모델임을 알 수 있습니다. 베이즈 정리가 이렇게도 활용되는 구나 하는 점과 단순한 모델의 강력함을 꼭 기억하고 넘어갔으면 좋겠습니다.